In [603]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from category_encoders import BinaryEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, make_scorer

##### Second Objective Dynamic Risk Management:
- The second objective of this project to use predictive analytics to predict total_claim for each policyholders.

In [555]:
insurance_claimsdf = pd.read_csv('C:/Users/chimi/Desktop/Python Data Science Projects/insurance_claim/data/insurance_claims.csv')

In [556]:
# Shape of the dataset
insurance_claimsdf.shape

(1000, 40)

In [557]:
# Display the first few rows and summary information about the dataset
display_info = {
    'head' : insurance_claimsdf.head(),
    'info' : insurance_claimsdf.info()  
    
}
display_info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_number                1000 non-null   int64  
 3   policy_bind_date             1000 non-null   object 
 4   policy_state                 1000 non-null   object 
 5   policy_csl                   1000 non-null   object 
 6   policy_deductable            1000 non-null   int64  
 7   policy_annual_premium        1000 non-null   float64
 8   umbrella_limit               1000 non-null   int64  
 9   insured_zip                  1000 non-null   int64  
 10  insured_sex                  1000 non-null   object 
 11  insured_education_level      1000 non-null   object 
 12  insured_occupation           1000 non-null   object 
 13  insured_hobbies    

{'head':    months_as_customer  age  policy_number policy_bind_date policy_state  \
 0                 328   48         521585       2014-10-17           OH   
 1                 228   42         342868       2006-06-27           IN   
 2                 134   29         687698       2000-09-06           OH   
 3                 256   41         227811       1990-05-25           IL   
 4                 228   44         367455       2014-06-06           IL   
 
   policy_csl  policy_deductable  policy_annual_premium  umbrella_limit  \
 0    250/500               1000                1406.91               0   
 1    250/500               2000                1197.22         5000000   
 2    100/300               2000                1413.14         5000000   
 3    250/500               2000                1415.74         6000000   
 4   500/1000               1000                1583.91         6000000   
 
    insured_zip  ... police_report_available total_claim_amount injury_claim  \
 0

In [558]:
# Columns of the dataframe
insurance_claimsdf.columns

Index(['months_as_customer', 'age', 'policy_number', 'policy_bind_date',
       'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital-gains', 'capital-loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported', '_c39'],
      dtype='object')

In [559]:
# Looking at number of unique policies
# All the policies are unique.
# Trying to see if they were any duplicate policies
insurance_claimsdf['policy_number'].nunique()

1000

In [560]:
# Drop all policies number - too many unique values
insurance_claimsdf.drop('policy_number', axis=1, inplace=True)

In [561]:
# Check for zipcode too
insurance_claimsdf['insured_zip'].value_counts()

insured_zip
477695    2
469429    2
446895    2
431202    2
456602    2
         ..
476303    1
450339    1
476502    1
600561    1
612260    1
Name: count, Length: 995, dtype: int64

In [562]:
# Drop the zipcode
insurance_claimsdf.drop('insured_zip', axis=1, inplace=True)

In [563]:
# Check for duplicates

duplicate_rows = insurance_claimsdf.duplicated().sum()

duplicate_rows

0

In [564]:
# Check for missing values
insurance_claimsdf.isnull().sum()

months_as_customer                0
age                               0
policy_bind_date                  0
policy_state                      0
policy_csl                        0
policy_deductable                 0
policy_annual_premium             0
umbrella_limit                    0
insured_sex                       0
insured_education_level           0
insured_occupation                0
insured_hobbies                   0
insured_relationship              0
capital-gains                     0
capital-loss                      0
incident_date                     0
incident_type                     0
collision_type                    0
incident_severity                 0
authorities_contacted            91
incident_state                    0
incident_city                     0
incident_location                 0
incident_hour_of_the_day          0
number_of_vehicles_involved       0
property_damage                   0
bodily_injuries                   0
witnesses                   

In [565]:
# Check for missing values in each column
missing_values = insurance_claimsdf.isnull().sum()

missing_values[missing_values > 0]

authorities_contacted      91
_c39                     1000
dtype: int64

In [566]:
# Calculate the mode of the 'authorities_contacted' column
mode_authorities_contacted = insurance_claimsdf['authorities_contacted'].mode()[0]

# Fill missing values in 'authorities_contacted' with the mode
insurance_claimsdf['authorities_contacted'].fillna(mode_authorities_contacted, inplace=True)

# Check if there are any remaining missing values in the column
insurance_claimsdf['authorities_contacted'].isnull().sum()

C:\Users\chimi\AppData\Local\Temp\ipykernel_1620\3010056018.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  insurance_claimsdf['authorities_contacted'].fillna(mode_authorities_contacted, inplace=True)


0

In [567]:
# The only column that has missing value is c39-all the 1000 rows are missing in this column
# Drop the column
insurance_claimsdf.drop(columns = ['_c39'], axis=1, inplace=True)

#### Feature Engineering subjects for this model
- Duration of Policy: Examining whether policies that have been kept for a long time may provide us with information about how the severity of claims may be impacted by more seasoned policyholders.
- Vehicle Age: Do older cars typically have more serious damage and cost more to repair?
- Day of Week: Does the severity of a claim change depending on the day of the occurrence particular driving behaviors are reflected in?
- Categorical Aggregations: Higher claim severities may be predicted by trends in categories such as occupation.
- Infrastructure and Environment: The local environment can significantly reduce the severity of events.Additionally, driving conditions may be impacted by seasonal patterns, incident times resulting from traffic, and seasonal patterns.

In [568]:
# Perform some light feature engineering relevant to this model which can impact total claim

insurance_claimsdf['incident_date'] = pd.to_datetime(insurance_claimsdf['incident_date'])
insurance_claimsdf['policy_bind_date'] = pd.to_datetime(insurance_claimsdf['policy_bind_date'])

# Calculate Time Since Policy Inception in days
insurance_claimsdf['time_since_policy_inception'] = (insurance_claimsdf['incident_date'] - insurance_claimsdf['policy_bind_date']).dt.days

# Calculate Age of Vehicle at the time of the incident
# Ensure that 'auto_year' is used correctly by subtracting from the incident date's year
insurance_claimsdf['vehicle_age_at_incident'] = insurance_claimsdf['incident_date'].dt.year - insurance_claimsdf['auto_year']

# Extract Day of the Week from incident_date (0 = Monday, 6 = Sunday)
insurance_claimsdf['day_of_week'] = insurance_claimsdf['incident_date'].dt.dayofweek

# Display the new columns to confirm creation
insurance_claimsdf[['time_since_policy_inception', 'vehicle_age_at_incident', 'day_of_week']].head()

,time_since_policy_inception,vehicle_age_at_incident,day_of_week
0,100,11,6
1,3130,8,2
2,5282,8,6
3,8996,1,5
4,256,6,1


In [569]:
# Function to determine the part of the day
def get_part_of_day(hour):
    if 6 <= hour <= 11:
        return 'Morning'
    elif 12 <= hour <= 17:
        return 'Afternoon'
    elif 18 <= hour <= 23:
        return 'Evening'
    else:
        return 'Night'

# Function to determine season
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

insurance_claimsdf['part_of_day'] = insurance_claimsdf['incident_hour_of_the_day'].apply(get_part_of_day)
insurance_claimsdf['incident_season'] = insurance_claimsdf['incident_date'].dt.month.apply(get_season)


In [570]:
insurance_claimsdf.head(5)

,months_as_customer,age,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,...,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,time_since_policy_inception,vehicle_age_at_incident,day_of_week,part_of_day,incident_season
0,328,48,2014-10-17,OH,250/500,1000,1406.91,0,MALE,MD,...,52080,Saab,92x,2004,Y,100,11,6,Night,Winter
1,228,42,2006-06-27,IN,250/500,2000,1197.22,5000000,MALE,MD,...,3510,Mercedes,E400,2007,Y,3130,8,2,Morning,Winter
2,134,29,2000-09-06,OH,100/300,2000,1413.14,5000000,FEMALE,PhD,...,23100,Dodge,RAM,2007,N,5282,8,6,Morning,Winter
3,256,41,1990-05-25,IL,250/500,2000,1415.74,6000000,FEMALE,PhD,...,50720,Chevrolet,Tahoe,2014,Y,8996,1,5,Night,Winter
4,228,44,2014-06-06,IL,500/1000,1000,1583.91,6000000,MALE,Associate,...,4550,Accura,RSX,2009,N,256,6,1,Evening,Winter


In [571]:
# Initialize encoders
label_encoder = LabelEncoder()
binary_encoder = BinaryEncoder()  # This will be used specifically for high-cardinality nominal variables

# Label Encoding for binary categorical variables
insurance_claimsdf['insured_sex'] = label_encoder.fit_transform(insurance_claimsdf['insured_sex'])
insurance_claimsdf['property_damage'] = label_encoder.fit_transform(insurance_claimsdf['property_damage'])
insurance_claimsdf['police_report_available'] = label_encoder.fit_transform(insurance_claimsdf['police_report_available'])

In [572]:

# Define the order and corresponding mapping manually
education_mapping = {
    'High School': 0,  # Assuming high school as the base level of formal education
    'College': 1,      # Typically refers to some college coursework but less than an associate's degree
    'Associate': 2,    # An associate's degree is generally considered above 'some college' but below a bachelor's degree
    'Masters': 3,      # A graduate level degree, following a bachelor's
    'JD': 4,           # Professional degree, often considered parallel to a master's in terms of years of study
    'MD': 5,           # Professional degree, represents a high level of specialization post-graduate
    'PhD': 6           # The highest academic degree awarded by universities in most fields
}

# Apply the mapping to the column
insurance_claimsdf['insured_education_level'] = insurance_claimsdf['insured_education_level'].map(education_mapping)

# Verify the mapping
print(insurance_claimsdf['insured_education_level'].head())

0    5
1    5
2    6
3    6
4    2
Name: insured_education_level, dtype: int64


In [573]:
print(insurance_claimsdf['insured_education_level'].value_counts())

insured_education_level
4    161
0    160
2    145
5    144
3    143
6    125
1    122
Name: count, dtype: int64


In [574]:
# Claim Severity label
insurance_claimsdf['incident_severity'].value_counts()

incident_severity
Minor Damage      354
Total Loss        280
Major Damage      276
Trivial Damage     90
Name: count, dtype: int64

In [575]:
# Define the order and corresponding manual mapping
severity_mapping = {
    'Trivial Damage': 0,   # Least severe
    'Minor Damage': 1,     # More severe than trivial but less than major
    'Major Damage': 2,     # More severe than minor
    'Total Loss': 3        # Most severe
}

# Apply the mapping to the incident_severity column
insurance_claimsdf['incident_severity'] = insurance_claimsdf['incident_severity'].map(severity_mapping)

# Check to ensure the mapping has been applied correctly
print(insurance_claimsdf[['incident_severity']].head())
print(insurance_claimsdf['incident_severity'].value_counts())

   incident_severity
0                  2
1                  1
2                  1
3                  2
4                  1
incident_severity
1    354
3    280
2    276
0     90
Name: count, dtype: int64


In [576]:
insurance_claimsdf['incident_severity'].value_counts()

incident_severity
1    354
3    280
2    276
0     90
Name: count, dtype: int64

In [577]:
# Initialize label encoder
label_encoder = LabelEncoder()

# Apply label encoding to each of the high cardinality nominal variables
insurance_claimsdf['insured_occupation'] = label_encoder.fit_transform(insurance_claimsdf['insured_occupation'])
insurance_claimsdf['insured_hobbies'] = label_encoder.fit_transform(insurance_claimsdf['insured_hobbies'])
insurance_claimsdf['incident_city'] = label_encoder.fit_transform(insurance_claimsdf['incident_city'])
insurance_claimsdf['auto_make'] = label_encoder.fit_transform(insurance_claimsdf['auto_make'])
insurance_claimsdf['auto_model'] = label_encoder.fit_transform(insurance_claimsdf['auto_model'])

In [578]:
# Use pd.get_dummies to encode these and update the dataframe
columns_to_encode = ['policy_state', 'policy_csl', 'insured_relationship', 'incident_type', 'collision_type', 
                     'authorities_contacted', 'incident_state', 'part_of_day', 'incident_season']
    
# Apply pd.get_dummies to encode these columns and update the dataframe
insurance_claimsdf = pd.get_dummies(insurance_claimsdf, columns=columns_to_encode, dtype=int)

# Display the DataFrame to verify the changes
print(insurance_claimsdf.head())


   months_as_customer  age policy_bind_date  policy_deductable  \
0                 328   48       2014-10-17               1000   
1                 228   42       2006-06-27               2000   
2                 134   29       2000-09-06               2000   
3                 256   41       1990-05-25               2000   
4                 228   44       2014-06-06               1000   

   policy_annual_premium  umbrella_limit  insured_sex  \
0                1406.91               0            1   
1                1197.22         5000000            1   
2                1413.14         5000000            0   
3                1415.74         6000000            0   
4                1583.91         6000000            1   

   insured_education_level  insured_occupation  insured_hobbies  ...  \
0                        5                   2               17  ...   
1                        5                   6               15  ...   
2                        6                  

In [579]:
# Dropping 'auto_model', 'auto_year' for too many unique values-/fraud_reported no needed this model
insurance_claimsdf.drop(columns=['policy_bind_date','auto_year','fraud_reported','incident_date','incident_location'], axis=1, inplace=True)

#### Base Modeling

In [580]:
X = insurance_claimsdf.drop('total_claim_amount', axis=1)  # Features
y = insurance_claimsdf['total_claim_amount']  # Target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("R² score:", r2)

MSE: 3866757.6388000012
RMSE: 1966.4072921955922
R² score: 0.9941897152455526


#### Run CV to find the best performing model 

#### Date Preparation:

In [593]:
# Features and target
X = insurance_claimsdf.drop('total_claim_amount', axis=1)
y = insurance_claimsdf['total_claim_amount']


#### Model Evalution:

In [594]:
def evaluate_model(model, X, y, num_folds=5):
    # Negative RMSE for cross-validation (cross_val_score returns negative values for RMSE)
    rmse = make_scorer(mean_squared_error, squared=False)
    scores = cross_val_score(model, X, y, scoring=rmse, cv=num_folds)
    return np.abs(np.mean(scores))  # Return the average RMSE across all folds


In [596]:
# Dictionary of models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'Elastic Net': ElasticNet(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(n_estimators=100),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100),
    'XGBoost': xgb.XGBRegressor(objective='reg:squarederror'),
    'SVR':SVR()
}

# Dictionary to store RMSE for each model
model_rmse = {}

# Evaluate each model
for name, model in models.items():
    rmse = evaluate_model(model, X, y)
    model_rmse[name] = rmse
    print(f"{name}: Average RMSE = {rmse}")

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame.from_dict(model_rmse, orient='index', columns=['Average RMSE'])
results_df.sort_values(by='Average RMSE', inplace=True)  # Sort by RMSE


Linear Regression: Average RMSE = 2.331097007499055e-10
Ridge: Average RMSE = 4.99053133059385e-07
Lasso: Average RMSE = 0.020445472613820386
Elastic Net: Average RMSE = 0.0021465941315441618
Decision Tree: Average RMSE = 2801.686416818045
Random Forest: Average RMSE = 1878.076452957826
Gradient Boosting: Average RMSE = 1044.8483759875294
XGBoost: Average RMSE = 1443.0709534204912
SVR: Average RMSE = 26920.637453833224


##### Based on the RMSE value, the best performing model is Linear Regression